In [ ]:
#pip install sacremoses

In [3]:
import torch


## TAKEN FROM GITHUB CODE OF NEUTRALIZING BIAS
## https://github.com/rpryzant/neutralizing-bias

# from https://spacy.io/api/annotation#section-dependency-parsing
RELATIONS = [
    'det', 'amod', 'nsubj', 'prep', 'pobj', 'ROOT', 
    'attr', 'punct', 'advmod', 'compound', 'acl', 'agent', 
    'aux', 'ccomp', 'dobj', 'cc', 'conj', 'appos', 'nsubjpass', 
    'auxpass', 'poss', 'nummod', 'nmod', 'relcl', 'mark', 
    'advcl', 'pcomp', 'npadvmod', 'preconj', 'neg', 'xcomp', 
    'csubj', 'prt', 'parataxis', 'expl', 'case', 'acomp', 'predet',
    'quantmod', 'dep', 'oprd', 'intj', 'dative', 'meta', 'csubjpass', 
    '<UNK>'
]
REL2ID = {x: i for i, x in enumerate(RELATIONS)}

# from https://spacy.io/api/annotation#section-pos-tagging
POS_TAGS = [
    'DET', 'ADJ', 'NOUN', 'ADP', 'NUM', 'VERB', 'PUNCT', 'ADV', 
    'PART', 'CCONJ', 'PRON', 'X', 'INTJ', 'PROPN', 'SYM',
    '<UNK>'
]
POS2ID = {x: i for i, x in enumerate(POS_TAGS)}

import nltk
import numpy as np

from nltk.parse.stanford import StanfordDependencyParser

class Featurizer:

    def __init__(self, tok2id={}, pad_id=0, lexicon_feature_bits=1):
        self.tok2id = tok2id
        self.id2tok = {x: tok for tok, x in tok2id.items()}
        self.pad_id = pad_id

        self.pos2id = POS2ID
        self.rel2id = REL2ID

        self.lexicons = {
            'assertives': self.read_lexicon('lexicons/assertives_hooper1975.txt'),
            'entailed_arg': self.read_lexicon('lexicons/entailed_arg_berant2012.txt'),
            'entailed': self.read_lexicon('lexicons/entailed_berant2012.txt'), 
            'entailing_arg': self.read_lexicon('lexicons/entailing_arg_berant2012.txt'), 
            'entailing': self.read_lexicon('lexicons/entailing_berant2012.txt'), 
            'factives': self.read_lexicon('lexicons/factives_hooper1975.txt'),
            'hedges': self.read_lexicon('lexicons/hedges_hyland2005.txt'),
            'implicatives': self.read_lexicon('lexicons/implicatives_karttunen1971.txt'),
            'negatives': self.read_lexicon('lexicons/negative_liu2005.txt'),
            'positives': self.read_lexicon('lexicons/positive_liu2005.txt'),
            'npov': self.read_lexicon('lexicons/npov_lexicon.txt'),
            'reports': self.read_lexicon('lexicons/report_verbs.txt'),
            'strong_subjectives': self.read_lexicon('lexicons/strong_subjectives_riloff2003.txt'),
            'weak_subjectives': self.read_lexicon('lexicons/weak_subjectives_riloff2003.txt')
        }
        self.lexicon_feature_bits = lexicon_feature_bits


    def get_feature_names(self):

        lexicon_feature_names = list(self.lexicons.keys())
        context_feature_names = [x + '_context' for x in lexicon_feature_names]
        pos_names = list(list(zip(*sorted(self.pos2id.items(), key=lambda x: x[1])))[0])
        rel_names = list(list(zip(*sorted(self.rel2id.items(), key=lambda x: x[1])))[0])

        return lexicon_feature_names + context_feature_names + pos_names + rel_names        

    def read_lexicon(self, fp):
        out = set([
            l.strip() for l in open(fp, errors='ignore') 
            if not l.startswith('#') and not l.startswith(';')
            and len(l.strip().split()) == 1
        ])
        return out


    def lexicon_features(self, words, bits=2):
        assert bits in [1, 2]
        if bits == 1:
            true = 1
            false = 0
        else:
            true = [1, 0]
            false = [0, 1]
    
        out = []
        for word in words:
            out.append([
                true if word in lexicon else false 
                for _, lexicon in self.lexicons.items()
            ])
        out = np.array(out)

        if bits == 2:
            out = out.reshape(len(words), -1)

        return out


    def context_features(self, lex_feats, window_size=2):
        out = []
        nwords = lex_feats.shape[0]
        nfeats = lex_feats.shape[1]
        for wi in range(lex_feats.shape[0]):
            window_start = max(wi - window_size, 0)
            window_end = min(wi + window_size + 1, nwords)

            left = lex_feats[window_start: wi, :] if wi > 0 else np.zeros((1, nfeats))
            right = lex_feats[wi + 1: window_end, :] if wi < nwords - 1 else np.zeros((1, nfeats))

            out.append((np.sum(left + right, axis=0) > 0).astype(int))

        return np.array(out)


    def features(self, id_seq, rel_ids, pos_ids):
        if self.pad_id in id_seq:
            pad_idx = id_seq.index(self.pad_id)
            pad_len = len(id_seq[pad_idx:])
            id_seq = id_seq[:pad_idx]
            rel_ids = rel_ids[:pad_idx]
            pos_ids = pos_ids[:pad_idx]
        else:
            pad_len = 0

        toks = [self.id2tok[x] for x in id_seq]
        # build list of [word, [tok indices the word came from]]
        words = []
        word_indices = []
        for i, tok in enumerate(toks):
            if tok.startswith('##'):
                words[-1] += tok.replace('##', '')
                word_indices[-1].append(i)
            else:
                words.append(tok)
                word_indices.append([i])

        # get expert features
        lex_feats = self.lexicon_features(words, bits=self.lexicon_feature_bits)
        context_feats = self.context_features(lex_feats)
        expert_feats = np.concatenate((lex_feats, context_feats), axis=1)
        # break word-features into tokens
        feats = np.concatenate([
            np.repeat(np.expand_dims(word_vec, axis=0), len(indices), axis=0) 
            for (word_vec, indices) in zip(expert_feats, word_indices)
        ], axis=0)

        # add in the pos and relational features
        pos_feats = np.zeros((len(pos_ids), len(POS2ID)))
        pos_feats[range(len(pos_ids)), pos_ids] = 1
        rel_feats = np.zeros((len(rel_ids), len(REL2ID)))
        rel_feats[range(len(rel_ids)), rel_ids] = 1
        
        feats = np.concatenate((feats, pos_feats, rel_feats), axis=1)

        # add pad back in                
        feats = np.concatenate((feats, np.zeros((pad_len, feats.shape[1]))))

        return feats


    def featurize_batch(self, batch_ids, rel_ids, pos_ids, padded_len=0):
        """ takes [batch, len] returns [batch, len, features] """
        print(rel_ids)
        print(batch_ids)
        print(pos_ids)
        batch_feats = [
            self.features(list(id_seq), list(rel_ids), list(pos_ids)) 
            for id_seq, rel_ids, pos_ids in zip(batch_ids, rel_ids, pos_ids)]
        batch_feats = np.array(batch_feats)
        return batch_feats

In [4]:
from transformers import AutoTokenizer, BertModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tok2id = tokenizer.vocab
tok2id['<del>'] = len(tok2id)
bertmodel = BertModel.from_pretrained("bert-base-uncased")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
layers = 12
hidden_size = 768

## TAKEN FROM GITHUB CODE OF NEUTRALIZING BIAS
## https://github.com/rpryzant/neutralizing-bias

class ConcatCombine(torch.nn.Module):
    def __init__(self, hidden_size, feature_size, out_size, layers,
            dropout_prob, small=False, pre_enrich=False, activation=False,
            include_categories=False, category_emb=False,
            add_category_emb=False):
        super(ConcatCombine, self).__init__()

        self.include_categories = include_categories
        self.add_category_emb = add_category_emb
        if include_categories:
            if category_emb and not add_category_emb:
                feature_size *= 2
            elif not category_emb:
                feature_size += 43

        if layers == 1:
            self.out = torch.nn.Sequential(
                torch.nn.Linear(hidden_size + feature_size, out_size),
                torch.nn.Dropout(dropout_prob))
        elif layers == 2:
            waist_size = min(hidden_size, feature_size) if small else max(hidden_size, feature_size)
            if activation:
                self.out = torch.nn.Sequential(
                    torch.nn.Linear(hidden_size + feature_size, waist_size),
                    torch.nn.Dropout(dropout_prob),
                    torch.nn.ReLU(),
                    torch.nn.Linear(waist_size, out_size),
                    torch.nn.Dropout(dropout_prob))
            else:
                self.out = torch.nn.Sequential(
                    torch.nn.Linear(hidden_size + feature_size, waist_size),
                    torch.nn.Dropout(dropout_prob),
                    torch.nn.Linear(waist_size, out_size),
                    torch.nn.Dropout(dropout_prob))
        if pre_enrich:
            if activation:
                self.enricher = torch.nn.Sequential(
                    torch.nn.Linear(feature_size, feature_size),
                    torch.nn.ReLU())
            else:
                self.enricher = torch.nn.Linear(feature_size, feature_size)
        else:
            self.enricher = None
        # manually set cuda because module doesn't see these combiners for bottom 
#         if CUDA:
#             self.out = self.out.cuda()
#             if self.enricher: 
#                 self.enricher = self.enricher.cuda()
                
    def forward(self, hidden, features, categories=None):
        if self.include_categories:
            categories = categories.unsqueeze(1)
            categories = categories.repeat(1, features.shape[1], 1)
            if self.add_category_emb:
                features = features + categories
            else:
                features = torch.cat((features, categories), -1)

        if self.enricher is not None:
            features = self.enricher(features)

        return self.out(torch.cat((hidden, features), dim=-1))
    

class BertDetector(torch.nn.Module):
    def __init__(self,cls_num_labels=2,token_num_labels=2,tok2id=None):
        super(BertDetector,self).__init__()
        self.bert = bertmodel
        self.featurizer = Featurizer(tok2id,lexicon_feature_bits=1)
        self.cls_dropout = torch.nn.Dropout(0.15)
        self.cls_classifier = torch.nn.Linear(hidden_size,cls_num_labels)
        
        self.token_dropout = torch.nn.Dropout(0.15)
        #self.token_classifier = torch.nn.Linear(hidden_size,token_num_labels)
        
        self.token_classifier = ConcatCombine(
                hidden_size, 90, token_num_labels, 
                1, 0.15, False, pre_enrich=False,
                activation=False,
                include_categories=False,
                category_emb=False,
                add_category_emb=False)
        
    def forward(self,input_ids,token_type_ids=None,attention_mask=None, 
        labels=None,rel_ids=None,pos_ids=None,categories=None,pre_len=None):
        features = torch.tensor(self.featurizer.featurize_batch(
            input_ids.numpy(), 
            rel_ids.numpy(), 
            pos_ids.numpy(), 
            padded_len=input_ids.shape[1]),dtype=torch.float)
        results = self.bert(input_ids)
        sequence = results.last_hidden_state
        pooled = results.pooler_output
        cls_logits = self.cls_dropout(self.cls_classifier(pooled))
        token_logits = self.token_dropout(self.token_classifier(sequence,features))
        return cls_logits,token_logits

In [14]:
text_1 = "China vowed on Wednesday to \"fight back\" after the United States announced a possible second round of tariff hikes on $200 billion worth of Chinese goods. The U.S. proposal released on Tuesday included increased taxes on imported food products and consumer electronics."
text_2 = "not sure"
indexed_tokens = tokenizer.encode(text_1, add_special_tokens=True)

tokens_tensor = torch.tensor([indexed_tokens])
lengths = (tokens_tensor == 102).nonzero(as_tuple=True)[1]
try: 
    segments_ids = [0] * (lengths[0] + 1) + [1] * (lengths[1] - lengths[0])
except:
    segments_ids = [0] * (lengths[0] + 1)
segments_tensors = torch.tensor([segments_ids])
segments_tensors
rel_tensor = torch.tensor([REL2ID[x] for x in ["<UNK>"]*len(indexed_tokens)])
pos_tensor = torch.tensor([POS2ID[x] for x in ["<UNK>"]*len(indexed_tokens)])

(tokens_tensor,segments_tensors,rel_tensor,pos_tensor)

(tensor([[  101,  2859, 18152,  2006,  9317,  2000,  1000,  2954,  2067,  1000,
           2044,  1996,  2142,  2163,  2623,  1037,  2825,  2117,  2461,  1997,
          23234, 21857,  2015,  2006,  1002,  3263,  4551,  4276,  1997,  2822,
           5350,  1012,  1996,  1057,  1012,  1055,  1012,  6378,  2207,  2006,
           9857,  2443,  3445,  7773,  2006, 10964,  2833,  3688,  1998,  7325,
           8139,  1012,   102]]),
 tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0]]),
 tensor([45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45,
         45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45,
         45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45]),
 tensor([15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
         15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,

In [15]:
model = BertDetector(tok2id=tok2id)
results = model(tokens_tensor,token_type_ids=segments_tensors,rel_ids=rel_tensor.unsqueeze(0),pos_ids=pos_tensor.unsqueeze(0))
(results,torch.nn.Softmax(dim=1)(results[1]))

[[45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45
  45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45
  45 45 45 45 45]]
[[  101  2859 18152  2006  9317  2000  1000  2954  2067  1000  2044  1996
   2142  2163  2623  1037  2825  2117  2461  1997 23234 21857  2015  2006
   1002  3263  4551  4276  1997  2822  5350  1012  1996  1057  1012  1055
   1012  6378  2207  2006  9857  2443  3445  7773  2006 10964  2833  3688
   1998  7325  8139  1012   102]]
[[15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
  15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
  15 15 15 15 15]]


((tensor([[ 0.7067, -0.7842]], grad_fn=<MulBackward0>),
  tensor([[[ 0.9394,  0.2291],
           [ 0.2739, -0.1885],
           [ 0.1409, -0.3158],
           [ 0.0000, -0.4369],
           [ 0.1004,  0.0000],
           [ 0.3477, -0.2110],
           [ 0.0000,  0.0000],
           [ 0.2604, -0.1540],
           [-0.0000, -0.0000],
           [ 0.0000,  0.0000],
           [ 0.6586, -0.0000],
           [ 0.0000, -0.1458],
           [ 1.2494, -0.0000],
           [ 0.9563,  0.0468],
           [ 0.0000, -0.0000],
           [-0.0810, -0.4441],
           [ 0.1827, -0.5043],
           [ 0.3458, -0.3584],
           [-0.1947, -0.4056],
           [ 0.0000, -0.6572],
           [ 0.7294,  0.0107],
           [ 0.1171,  0.0090],
           [ 0.0000, -0.0000],
           [ 0.5165, -0.5907],
           [ 0.4840, -0.3296],
           [ 0.9114, -0.0000],
           [ 0.0403, -0.5425],
           [ 0.0000, -0.3294],
           [ 0.6837, -0.5323],
           [ 0.3622, -0.3345],
           [ 0

In [42]:
from simplediff import diff

In [ ]:
diff(biased.strip().split(),nonbiased.strip().split())

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

In [66]:
text_test= "165188319	ch ##lor ##of ##or ##m \" the molecular life ##sa ##ver \" an article at oxford university providing interesting facts about ch ##lor ##of ##or ##m .	ch ##lor ##of ##or ##m \" the molecular life ##sa ##ver \" an article at oxford university providing facts about ch ##lor ##of ##or ##m .	chloroform \"the molecular lifesaver\" an article at oxford university providing interesting facts about chloroform.	chloroform \"the molecular lifesaver\" an article at oxford university providing facts about chloroform.	NOUN NOUN NOUN NOUN NOUN PUNCT DET ADJ NOUN NOUN NOUN PUNCT DET NOUN ADP NOUN NOUN VERB ADJ NOUN ADP NOUN NOUN NOUN NOUN NOUN PUNCT	ROOT ROOT ROOT ROOT ROOT punct det amod dobj dobj dobj punct det appos prep compound pobj acl amod dobj prep pobj pobj pobj pobj pobj punct"
[revid, _, _, biased, nonbiased, pos, rels] = text_test.strip().split("\t")

indexed_tokens = tokenizer.encode(biased.strip(),nonbiased.strip(), add_special_tokens=True)

tokens_tensor = torch.tensor([indexed_tokens])
lengths = (tokens_tensor == 102).nonzero(as_tuple=True)[1]
try: 
    segments_ids = [0] * (lengths[0] + 1) + [1] * (lengths[1] - lengths[0])
except:
    segments_ids = [0] * (lengths[0] + 1)
segments_tensors = torch.tensor([segments_ids])
segments_tensors
rel_tensor = torch.tensor([REL2ID[x] for x in rels.strip().split(" ")]*2)
pos_tensor = torch.tensor([POS2ID[x] for x in pos.strip().split(" ")]*2)
while rel_tensor.size()[0] < len(indexed_tokens):
    rel_tensor = torch.cat((rel_tensor, torch.tensor([0])), dim=-1)
while pos_tensor.size()[0] < len(indexed_tokens):
    pos_tensor = torch.cat((pos_tensor, torch.tensor([0])), dim=-1)
    
ground_truth_list = [(x[0],len(x[1])) for x in diff(biased.strip().split(),nonbiased.strip().split())]
ground_truth = []
for i in ground_truth_list:
    if i[0] == "-":
        ground_truth.extend([1]*i[1])
    else:
        ground_truth.extend([0]*i[1])
ground_truth
#rel_tensor = torch.tensor([REL2ID[x] for x in ["<UNK>"]*len(indexed_tokens)])
#pos_tensor = torch.tensor([POS2ID[x] for x in ["<UNK>"]*len(indexed_tokens)])

(tokens_tensor,segments_tensors,rel_tensor,pos_tensor)

(tensor([[  101, 10381, 10626, 11253,  2953,  2213,  1000,  1996,  8382,  2166,
           3736,  6299,  1000,  2019,  3720,  2012,  4345,  2118,  4346,  5875,
           8866,  2055, 10381, 10626, 11253,  2953,  2213,  1012,   102, 10381,
          10626, 11253,  2953,  2213,  1000,  1996,  8382,  2166,  3736,  6299,
           1000,  2019,  3720,  2012,  4345,  2118,  4346,  8866,  2055, 10381,
          10626, 11253,  2953,  2213,  1012,   102]]),
 tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1]]),
 tensor([ 5,  5,  5,  5,  5,  7,  0,  1, 14, 14, 14,  7,  0, 17,  3,  9,  4, 10,
          1, 14,  3,  4,  4,  4,  4,  4,  7,  5,  5,  5,  5,  5,  7,  0,  1, 14,
         14, 14,  7,  0, 17,  3,  9,  4, 10,  1, 14,  3,  4,  4,  4,  4,  4,  7,
          0,  0]),
 tensor([2, 2, 2, 2, 2, 6, 0, 1, 2, 2, 2, 6, 0, 2, 3, 2, 2, 5, 1, 2, 3, 2, 2, 2,
 

In [67]:
(tokens_tensor.size(),segments_tensors.size(),rel_tensor.size(),pos_tensor.size())

(torch.Size([1, 56]), torch.Size([1, 56]), torch.Size([56]), torch.Size([56]))

In [68]:
model = BertDetector(tok2id=tok2id)
results = model(tokens_tensor,token_type_ids=segments_tensors,rel_ids=rel_tensor.unsqueeze(0),pos_ids=pos_tensor.unsqueeze(0))
(results,torch.nn.Softmax(dim=1)(results[1]),torch.argmax(torch.nn.Softmax(dim=1)(results[1]),dim=-1))

[[ 5  5  5  5  5  7  0  1 14 14 14  7  0 17  3  9  4 10  1 14  3  4  4  4
   4  4  7  5  5  5  5  5  7  0  1 14 14 14  7  0 17  3  9  4 10  1 14  3
   4  4  4  4  4  7  0  0]]
[[  101 10381 10626 11253  2953  2213  1000  1996  8382  2166  3736  6299
   1000  2019  3720  2012  4345  2118  4346  5875  8866  2055 10381 10626
  11253  2953  2213  1012   102 10381 10626 11253  2953  2213  1000  1996
   8382  2166  3736  6299  1000  2019  3720  2012  4345  2118  4346  8866
   2055 10381 10626 11253  2953  2213  1012   102]]
[[2 2 2 2 2 6 0 1 2 2 2 6 0 2 3 2 2 5 1 2 3 2 2 2 2 2 6 2 2 2 2 2 6 0 1 2
  2 2 6 0 2 3 2 2 5 1 2 3 2 2 2 2 2 6 0 0]]


((tensor([[-0.1879,  0.2159]], grad_fn=<MulBackward0>),
  tensor([[[-0.9896, -0.5885],
           [-0.0000, -0.1043],
           [-0.9724, -0.7916],
           [-0.7461, -0.0000],
           [-0.0000, -0.3750],
           [-0.0000, -0.1544],
           [ 0.0614, -0.4089],
           [-0.2645, -0.0779],
           [-0.6189, -0.0378],
           [-0.6264, -0.0609],
           [-0.0000, -0.7182],
           [-0.8381, -0.4092],
           [ 0.2552, -0.3033],
           [-0.0000, -0.0000],
           [-0.6066, -0.2585],
           [-0.1947, -0.0383],
           [-0.0000, -0.0000],
           [-0.0000, -0.0000],
           [-0.1722, -0.0000],
           [-0.3849, -0.3014],
           [-0.0000, -0.1335],
           [-0.7658, -0.2917],
           [-0.2750, -0.0000],
           [-0.0000, -0.0000],
           [-0.4523, -0.0000],
           [-0.8212, -0.5299],
           [-0.0000, -0.0000],
           [ 0.1374, -0.2520],
           [-0.3398,  0.7106],
           [-0.0000, -0.5433],
           [-0

In [69]:
x = np.array(results[1].detach())[:,:,:2]
x = x - x.max(axis=2, keepdims=True)
y = np.exp(x)
y / y.sum(axis=2, keepdims=True)
y[:,:,-1]

array([[1.        , 0.9009634 , 1.        , 1.        , 0.6872803 ,
        0.8569493 , 0.62477714, 1.        , 1.        , 1.        ,
        0.48761815, 1.        , 0.57204586, 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        0.8749847 , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 0.67752767, 1.        , 0.58082134,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        0.5394351 , 1.        , 1.        , 0.98471063, 1.        ,
        1.        , 0.730942  , 1.        , 1.        , 0.49842423,
        1.        , 0.5844139 , 0.5814122 , 1.        , 0.54270226,
        1.        ]], dtype=float32)